<!--html_preserve-->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-130562131-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-130562131-1');
</script>
<!--/html_preserve-->


In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/uq1.jpg")



# Introduction{-}

This tutorial introduces network analysis using R. Network analysis is a method for visualization that can be used to represent various types of data. In addition to being a visualization technique, networks have certain statistical properties that can be compared which makes network analysis a very useful procedure. To this end, this tutorial shows how to create and modify network graphs. The entire R markdown document for the sections below can be downloaded [here](https://slcladal.github.io/net.Rmd). This tutorial builds on [a tutorial on plotting collocation networks](https://corpling.hypotheses.org/3341) by [Guillaume Desagulier](http://www2.univ-paris8.fr/desagulier/home/), a tutorial on network analysis by offered by Alice Miller from the [Digital Observatory](https://www.qut.edu.au/institute-for-future-environments/facilities/digital-observatory/about-do) at the [Queensland University of Technology](https://www.qut.edu.au/), and [this tutorial](https://tm4ss.github.io/docs/Tutorial_5_Co-occurrence.html) by [Andreas Niekler](http://asv.informatik.uni-leipzig.de/staff/Andreas_Niekler) and [Gregor Wiedemann](https://www.hans-bredow-institut.de/en/staff/gregor-wiedemann).   

>
> **How can you display the relationship between different elements, be they authors, characters, or words?**
>

The most common way to visualize such relationships are networks [@silge2017text : 131-137]. Networks, also called graphs, consist of nodes (typically represented as dots) and edges (typically represented as lines) and they can be directed or undirected networks. 

In directed networks, the direction of edges is captured. For instance, the exports of countries. In such cases the lines are directed and typically have arrows to indicate direction. The thickness of lines can also be utilized to encode information such as frequency of contact.


The example that we will be concerned with focuses on the first type of data as it is by far the most common way in which relationships are coded.To show how to create a network, we will have a look at the network that the characters in William Shakespeare's *Romeo and Juliet* form.

## Preparation and session set up{-}

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/intror.html). For this tutorials, we need to install certain *packages* from an R *library* so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead and ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# set options
options(stringsAsFactors = F)         # no automatic data transformation
options("scipen" = 100, "digits" = 4) # suppress math annotation
# install libraries
install.packages(c("GGally", "network", "sna", "tidyverse", "knitr", 
                   "kableExtra", "gutenbergr", "tm", "Matrix"))


In [ ]:
# activate packages
library(knitr)
library(kableExtra)
library(DT)


Once you have installed R, RStudio, and have also initiated the session by executing the code shown above, you are good to go.

# Network Networks

This section shows how to create a network visualization with the `network` and the `GGally` packages. The network we will generate shows how often characters in William Shakespeare's *Romeo and Juliet* occurred in the same scene. In a first step, we load the necessary packages from the library.


In [ ]:
# activate packages
library(GGally)
library(network)


The issue we want to investigate here relates to networks of personas in Shakespeare's *Romeo and Juliet* and we thus load this famous work of fiction. Now that we have loaded the data, we need to split the data into scenes. Scenes during which personas leave or enter will have to be split too so that we arrive at a table that contains the personas that are present during a sub-scene.



In [ ]:
# load data
rom <- read.delim("https://slcladal.github.io/data/romeo_tidy.txt", sep = "\t")
rome <- crossprod(table(rom[1:2]))
diag(rome) <- 0
romeo <- as.data.frame(rome)
romeo[1:5, 1:5]


The data shows how often a character has appeared with each other character in the play - only Friar Lawrence and Friar John were excluded because they only appear in one scene where they talk to each other. 

In a next step, we create a network object from the matrix. In addition, we define vertex names as these will be used as labels in the network plot.


In [ ]:
net = network::network(romeo,
                       directed = FALSE,
                       ignore.eval = FALSE,
                       names.eval = "weights")
# vertex names
network.vertex.names(net) = rownames(romeo)
net


Unfortunately, network object are somewhat obscure in that they can not be displayed as simple data frames. 

However, this does not matter at this point and we continue by visualizing the network in a simple network plot using the ggnet2 function.


In [ ]:
ggnet2(net, 
       size = 6, 
       color = "goldenrod", 
       edge.size = .5,
       edge.color = "lightgrey", 
       label = TRUE, 
       label.size = 3)


The basic network shown above only shows who has co-occurred with whom but it is not really informative. Therefore, we add information to the network object. 

To add information to a simple network graph, we create a new variable. This variable shows to which family each of the characters belong. To do this, we create a separate vector for each family which holds the characters that are members of this family. Then, we create a variable in the network object called "Family" which represents the family which the characters belong to. 


In [ ]:
# define colors (by family)
mon <- c("ABRAM", "BALTHASAR", "BENVOLIO", "LADY MONTAGUE", "MONTAGUE", "ROMEO")
cap <- c("CAPULET", "CAPULET’S COUSIN", "FIRST SERVANT", "GREGORY", "JULIET", 
         "LADY CAPULET", "NURSE", "PETER", "SAMPSON", "TYBALT")
oth <- c("APOTHECARY", "CHORUS", "FIRST CITIZEN", "FIRST MUSICIAN", "FIRST WATCH", 
         "FRIAR JOHN" , "FRIAR LAWRENCE", "MERCUTIO", "PAGE", "PARIS", "PRINCE", 
         "SECOND MUSICIAN", "SECOND SERVANT", "SECOND WATCH", "SERVANT", "THIRD MUSICIAN")
# add color by group
net %v% "Family" = ifelse(network.vertex.names(net) %in% cap, "Capulet", 
                   ifelse(network.vertex.names(net) %in% mon, "Montague", "other"))


We can now use the family variable to define a color so that each character can be associated with his or her family by coloring. In addition, we specify the edge attributes so that the thickness and the type of a line represent how often characters have co-occurred. Characters that co-occur frequently are connected by thick, straight, solid lines whereas characters that co-occurred together only infrequently are connected by thin, dotted lines.



In [ ]:
# modify color
net %v% "color" = ifelse(net %v% "Family" == "Capulet", "goldenrod", 
                  ifelse(net %v% "Family" == "Montague", "indianred4","gray60"))
# rescale edge size
network::set.edge.attribute(net, "weights", ifelse(net %e% "weights" <= 1, 0.25, 
                                   ifelse(net %e% "weights" <= 3, .5, 1)))
# define line type
network::set.edge.attribute(net, "lty", ifelse(net %e% "weights" == 0.25, 3, 
                                      ifelse(net %e% "weights" == .5, 2, 1)))


We now visualize the network again but include information such as who belongs to which family (the translucent circles around the names) and how often they have co-occurred (thickness any type of the lines). We also specify alpha values which make lines and circles more or less translucent (the higher the value, the more translucent are the lines and circles).



In [ ]:
ggnet2(net, 
       color = "color", 
       label = TRUE, 
       label.size = 3,
       alpha = 0.2,
       size = "degree",
       size.cut = 3,
       edge.size = "weights",
       edge.lty = "lty",
       edge.alpha = 0.5) +
  guides(color = FALSE, size = FALSE)


We have now plotted the network of characters in Shakespeare's *Romeo and Juliet* and we have added additional information to this plot. The characters are shown by name and the color behind their name informs us about which family they belong to (Capulet = golden, Montague = red, other = gray). The linetype and line thickness both provide information on how frequently characters interacted: slim dotted gray lines indicate that the characters only co-occurred once, dashed gray lines indicate that the characters co-occurred up to three times and thick solid gray lines show that the characters co-occured more than 3 times together. 

# Quanteda Networks 

The `quanteda` package contains many very useful functions for analyzing texts. Among these functions is the `textplot_network` function which provides a very handy way to display networks. The advantage of the network plots provided by or generated with the `quanteda` package is that you can create them with very little code. However, this comes at a cost as  these visualizations cannot be modified easily (which means that their design is not very flexible compared to other methods for generating network visualizations).    

In a first step, we transform the text vectors of the `romeo` data into a document-feature matrix using the `dfm ` function. 


In [ ]:
# load package
library(quanteda)
# create a document feature matrix
romeo_dfm <- quanteda::as.dfm(romeo)
# create feature co-occurrence matrix
romeo_fcm <- quanteda::fcm(romeo_dfm)
# inspect data
head(romeo_fcm)


This feature-co-occurrence matrix can then serve as the input for the `textplot_network` function which already generates a nice network graph. The network graph can then be modified or customized easily by defining the arguments of the `textplot_network` function. To see how and which arguments can be modified, you can use `?textplot_network`.



In [ ]:
quanteda::textplot_network(romeo_fcm, 
                           min_freq = 10, 
                           edge_alpha = 0.2, 
                           edge_color = "orange",
                           edge_size = 5)


# Tidy Networks

A great way to generate network graphs is to combine functions from the `igraph`, the `ggraph`, and the `tidygraph` packages. The advantages are that the syntax of for creating the networks aligns with the tidyverse style of writing R and that the graphs can eb modified very easily.

To generate network graphs in this way, we first load the required packages.


In [ ]:
library(igraph)
library(ggraph)
library(tidygraph)
library(tidyverse)


In a next step, we define the *nodes* and we can also add information about the odes that we can use later on (suh as frequency information).



In [ ]:
va <- romeo %>%
  dplyr::mutate(Persona = rownames(.),
                Occurrences = rowSums(.)) %>%
  dplyr::select(Persona, Occurrences) %>%
  dplyr::filter(!str_detect(Persona, "SCENE"))


In [ ]:
DT::datatable(va, rownames = F, filter="none", 
              caption = "Personas and their frequencies of occurrence in Shakespeare's *Romeo and Juliet*.", 
              options = list(pageLength = 10, scrollX=T))


Now, we define the *edges*, i.e., the connections between nodes and, again, we can add information in separate variables that we can use later on. 



In [ ]:
ed <- romeo %>%
  dplyr::mutate(from = rownames(.)) %>%
  tidyr::gather(to, Frequency, BALTHASAR:TYBALT) %>%
  dplyr::mutate(Frequency = ifelse(Frequency == 0, NA, Frequency))


In [ ]:
DT::datatable(ed, rownames = F, filter="none", 
              caption = "Edges between personasin Shakespeare's *Romeo and Juliet*.", 
              options = list(pageLength = 10, scrollX=T))


Now that we have generated tables for the edges and the nodes, we can generate a graph object.



In [ ]:
ig <- graph_from_data_frame(d=ed, vertices=va, directed = FALSE)



We will also add labels to the nodes as follows:



In [ ]:
tg <- tidygraph::as_tbl_graph(ig) %>% 
  activate(nodes) %>% 
  mutate(label=name)


Now, we use the number of occurrences to define Vertice size (or node size)



In [ ]:
v.size <- V(tg)$Occurrences
# inspect
v.size


We can also use the frequency information to define weights



In [ ]:
E(tg)$weight <- E(tg)$Frequency
# inspect weights
head(E(tg)$weight, 10)


Finally, we define colors (by family).



In [ ]:
# define colors (by family)
mon <- c("ABRAM", "BALTHASAR", "BENVOLIO", "LADY MONTAGUE", "MONTAGUE", "ROMEO")
cap <- c("CAPULET", "CAPULET’S COUSIN", "FIRST SERVANT", "GREGORY", "JULIET", "LADY CAPULET", "NURSE", "PETER", "SAMPSON", "TYBALT")
oth <- c("APOTHECARY", "CHORUS", "FIRST CITIZEN", "FIRST MUSICIAN", "FIRST WATCH", "FRIAR JOHN" , "FRIAR LAWRENCE", "MERCUTIO", "PAGE", "PARIS", "PRINCE", "SECOND MUSICIAN", "SECOND SERVANT", "SECOND WATCH", "SERVANT", "THIRD MUSICIAN")
# create color vectors
Family <- dplyr::case_when(names(evcent(tg)$vector) %in% mon ~ "MONTAGUE",
                              names(evcent(tg)$vector) %in% cap ~ "CAPULET",
                              TRUE ~ "Other")
# inspect colors
Family


Now, that we c´have created the different objects and defined their properties, we visualize the network.



In [ ]:
# set seed
set.seed(12345)
# edge size shows frequency of co-occurrence
tg %>%
   ggraph(layout = "fr") +
   geom_edge_arc(colour= "gray50",
                  lineend = "round",
                 strength = .1,
                 aes(edge_width = weight,
                     alpha = weight)) +
   geom_node_point(size=log(v.size)*2, 
                   aes(color=Family)) +
   geom_node_text(aes(label = name), 
                  repel = TRUE, 
                  point.padding = unit(0.2, "lines"), 
                  size=sqrt(v.size), 
                  colour="gray10") +
  scale_edge_width(range = c(0, 2.5)) +
  scale_edge_alpha(range = c(0, .3)) +
  theme_graph(background = "white") +
  theme(legend.position = "top") +
  guides(edge_width = FALSE,
         edge_alpha = FALSE)


# iGraph Networks 

@WN17 have written a very recommendable tutorial on co-occurrence analysis and they propose an alternative for generating complex network visualization for co-occurrences. Their approach is to create and customize a graph object based on the `iGraph` package. To see how to create sophisticated network graphs using the `iGraph` package, see [this tutorial](https://slcladal.github.io/coll.html) on analyzing collocations or  [this tutorial](https://tm4ss.github.io/docs/Tutorial_5_Co-occurrence.html). 


We have reached the end of this tutorial and you now know how to create and modify networks in R and how you can highlight aspects of your data. 


# Citation & Session Info {-}

Schweinberger, Martin. `r format(Sys.time(), '%Y')`. *Network Analysis using R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/net.html (Version `r format(Sys.time(), '%Y.%m.%d')`).


In [ ]:
@manual{schweinberger`r format(Sys.time(), '%Y')`net,
  author = {Schweinberger, Martin},
  title = {Network Analysis using R},
  note = {https://slcladal.github.io/net.html},
  year = {`r format(Sys.time(), '%Y')`},
  organization = "The University of Queensland, Australia. School of Languages and Cultures},
  address = {Brisbane},
  edition = {`r format(Sys.time(), '%Y.%m.%d')`}
}


In [ ]:
sessionInfo()



***

[Back to top](#introduction)

[Back to HOME](https://slcladal.github.io/index.html)

***

# References {-}
